사용법: 

Roboflow에서 데이터셋 Export 옵션에서 YOLO v5 Pytorch로 받은 후, 압축폴더를 원하는 directory에다가 해제. 
box_to_keypoin.ipynb 파일을 data 폴더와 같은 디렉토리에 넣은 후 실행. 

In [ ]:
import json
import os
import cv2 as cv
import matplotlib.pyplot as plt
import re

In [99]:
yml_path = './data/train/data.yaml'
num_points = 12

with open(yml_path, 'r') as fp:
    raw = fp.readlines()
    # print(raw[5])
    result = re.findall("'.+'", raw[5])
    labels = [i.replace('\'', '').strip() for i in result[0].split(',')]
    print(labels)

if num_points != len(labels):
    print("라벨 숫자 안 맞음 ")

['cat', 'left_ear', 'left_front_leg_bottom', 'left_front_leg_upper', 'left_hind_leg_bottom', 'left_hind_leg_upper', 'neck', 'nose', 'right_ear', 'right_front_leg_bottom', 'right_front_leg_upper', 'right_hind_leg_bottom', 'right_hind_leg_upper', 'spine', 'spine_middle', 'stomach', 'tail_0', 'tail_1', 'tail_2', 'tail_3', 'tail_4', 'tail_5']
라벨 숫자 안 맞음 


In [100]:
label_dict = {i: labels[i] for i in range(1, len(labels))}

In [111]:
# keypoint좌표 자동 변경
# keypoint 형식 = [x좌표, y좌표, 라벨번호, visibility(0 or 1)]
def converter(file_labels:str, file_image:str):
    keypoints = []
    img = cv.imread(file_image)
    img_w, img_h = img.shape[1], img.shape[0]

    with open(file_labels) as f:
        lines_txt = f.readlines()
        lines = []
        for line in lines_txt:
            lines.append([int(line.split()[0])] + [round(float(coord), 5) for coord in line.split()[1:]])

    for line in lines:
        if line[0] == 0:
            x_c = round(line[1] * img_w)
            y_c = round(line[2] * img_h)
            w = round(line[3] * img_w)
            h = round(line[4] * img_h)
            
            bboxes = ([round(x_c - w/2), round(y_c - h/2), round(x_c + w/2), round(y_c + h/2)])
        
        else:
            kp_id = line[0]
            x_c = round(line[1] * img_w)
            y_c = round(line[2] * img_h)
            keypoints.append([x_c, y_c, kp_id, 1]) # 기본으로 visibility는 1로 체크

    keypoints = sorted(keypoints, key=lambda x: x[2]) # keypoint id기준으로 상향정렬

    return bboxes, keypoints


# 사진+keypoint+bbox 시각화
def visualize(image_path:str, keypoints:list, bboxes:list, save=False):
    top_left_corner, bottom_right_corner = tuple([bboxes[0], bboxes[1]]), tuple([bboxes[2], bboxes[3]])
    image = cv.imread(image_path)
    img = cv.rectangle(image, top_left_corner, bottom_right_corner, (0, 255, 0), 3)
    
    for kp_idx, kp in enumerate(keypoints):
        center = tuple([kp[0], kp[1]])
        img = cv.circle(img, center, 5, (255,0,0), 5)
        img = cv.putText(img, " " + label_dict[kp[2]], center, cv.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 2)
    
    plt.figure(figsize=(15, 15))
    plt.imshow(img)


def dump2json(bboxes:list, keypoints: list, json_path: str):
    annotations = {}
    annotations['bboxes'], annotations['keypoints'] = bboxes, keypoints
    with open(json_path, "w") as f:
        json.dump(annotations, f)


def main(IMAGE_PATH:str, LABELS_PATH:str, ANNOTATIONS_PATH:str):
    

    image_file_list = [file.split('.jpg')[0] for file in os.listdir(IMAGE_PATH)]
    
    try:
        for i in image_file_list:
            label_path = os.path.join(LABELS_PATH, i + ".txt")
            image_path = os.path.join(IMAGE_PATH, i + ".jpg")
            bboxes, keypoints = converter(label_path, image_path)
            dump2json(bboxes, keypoints, os.path.join(ANNOTATIONS_PATH, i + '.json'))
        else:
            print("All Converted")
    except:
        "Error!"
        
    

In [112]:
IMAGE_PATH = './data/train/images/'
LABELS_PATH = './data/train/labels/'
ANNOTATIONS_PATH = './data/train/annotations/'

# annotations 폴더 생성
try:
    if not os.path.exists(ANNOTATIONS_PATH ):
        os.makedirs(ANNOTATIONS_PATH )
except OSError:
    print("Error while creating the data directory")

All Converted


In [ ]:
# 화이팅!
main(IMAGE_PATH, LABELS_PATH, ANNOTATIONS_PATH)